Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

from datetime import datetime
#импорт модели
from model import hyperparameters
from orion.primitives.tadgan import TadGAN

from orion.primitives.tadgan import score_anomalies

from utils import plot, plot_ts, plot_rws, plot_error, unroll_ts

ModuleNotFoundError: No module named 'utils'

Загрузка даных с датчика и запись названия всех датчиков в список col_names для возмоности поиска аномалий со всех датчиков единоразово

In [ ]:
project_data = pd.read_csv('results.csv')
col_names = []
for col_name in project_data.columns[1:-1]:
    col_names.append(col_name)

NameError: name 'pd' is not defined

Преобразование датафрейма в вид со значениями датчика и временем снятия данных с датчика

In [ ]:
sensor_name = col_names[1]
project_data[sensor_name] = pd.to_numeric(project_data[sensor_name], errors='coerce')
project_data = project_data[["timestamp", sensor_name]]
project_data.head()

Преобразование столбца с временем в столец с секундами, прошедшими от съёма первых данных

In [ ]:
# Две строки с датами
date1_str = project_data['timestamp'][0]
for i in range(project_data.shape[0]):
    date2_str = project_data['timestamp'].iloc[i]

    # Преобразование строк в datetime объекты
    date1 = datetime.strptime(date1_str, "%Y-%m-%d %H:%M:%S")
    date2 = datetime.strptime(date2_str, "%Y-%m-%d %H:%M:%S")

    # Перевод в UNIX-время (в секундах с 1 января 1970 года)
    timestamp1 = int(date1.timestamp())
    timestamp2 = int(date2.timestamp())

    # Разница во времени в секундах
    difference_in_seconds = timestamp2 - timestamp1
    project_data['timestamp'][i] = difference_in_seconds

# Вывод результата
print(f"Timestamp 1: {timestamp1} секунд {date1_str}")
print(f"Timestamp 2: {timestamp2} секунд{date2_str}")
print(f"Разница: {difference_in_seconds} секунд")


Получение двух массивов с данными и временем для дальнейшей обработки и создание датафрейма из этих данных

In [ ]:
a = project_data['timestamp'].tolist()
b = project_data[sensor_name].tolist()

data = {'timestamp': a, 'value': b}
df = pd.DataFrame(data)

Функция агрегирует значения за заданный промежуток времени, то есть собирает в массив данные с промежутком во времени равным переданному значению

In [ ]:
def time_segments_aggregate(X, interval, time_column, method=['mean']):
    """Aggregate values over given time span.
    Args:
        X (ndarray or pandas.DataFrame):
            N-dimensional sequence of values.
        interval (int):
            Integer denoting time span to compute aggregation of.
        time_column (int):
            Column of X that contains time values.
        method (str or list):
            Optional. String describing aggregation method or list of strings describing multiple
            aggregation methods. If not given, `mean` is used.
    Returns:
        ndarray, ndarray:
            * Sequence of aggregated values, one column for each aggregation method.
            * Sequence of index values (first index of each aggregated segment).
    """
    if isinstance(X, np.ndarray):
        X = pd.DataFrame(X)

    X = X.sort_values(time_column).set_index(time_column)

    if isinstance(method, str):
        method = [method]

    start_ts = X.index.values[0]
    max_ts = X.index.values[-1]

    values = list()
    index = list()
    while start_ts <= max_ts:
        end_ts = start_ts + interval
        subset = X.loc[start_ts:end_ts - 1]
        aggregated = [
            getattr(subset, agg)(skipna=True).values
            for agg in method
        ]
        values.append(np.concatenate(aggregated))
        index.append(start_ts)
        start_ts = end_ts

    return np.asarray(values), np.asarray(index)

X, index = time_segments_aggregate(df, interval=60, time_column='timestamp')


Заполняем пропуски данных и масштабируем (нормализируем) данные для последующей обработки

In [ ]:
imp = SimpleImputer()
X = imp.fit_transform(X)
scaler = MinMaxScaler(feature_range=(-1, 1))
X = scaler.fit_transform(X)
plot_ts(X)



Применение скользящего окна на наши данные

In [ ]:
def rolling_window_sequences(X, index, window_size, target_size, step_size, target_column,
                             drop=None, drop_windows=False):
    """Create rolling window sequences out of time series data.
    The function creates an array of input sequences and an array of target sequences by rolling
    over the input sequence with a specified window.
    Optionally, certain values can be dropped from the sequences.
    Args:
        X (ndarray):
            N-dimensional sequence to iterate over.
        index (ndarray):
            Array containing the index values of X.
        window_size (int):
            Length of the input sequences.
        target_size (int):
            Length of the target sequences.
        step_size (int):
            Indicating the number of steps to move the window forward each round.
        target_column (int):
            Indicating which column of X is the target.
        drop (ndarray or None or str or float or bool):
            Optional. Array of boolean values indicating which values of X are invalid, or value
            indicating which value should be dropped. If not given, `None` is used.
        drop_windows (bool):
            Optional. Indicates whether the dropping functionality should be enabled. If not
            given, `False` is used.
    Returns:
        ndarray, ndarray, ndarray, ndarray:
            * input sequences.
            * target sequences.
            * first index value of each input sequence.
            * first index value of each target sequence.
    """
    out_X = list()
    out_y = list()
    X_index = list()
    y_index = list()
    target = X[:, target_column]

    if drop_windows:
        if hasattr(drop, '__len__') and (not isinstance(drop, str)):
            if len(drop) != len(X):
                raise Exception('Arrays `drop` and `X` must be of the same length.')
        else:
            if isinstance(drop, float) and np.isnan(drop):
                drop = np.isnan(X)
            else:
                drop = X == drop

    start = 0
    max_start = len(X) - window_size - target_size + 1
    while start < max_start:
        end = start + window_size

        if drop_windows:
            drop_window = drop[start:end + target_size]
            to_drop = np.where(drop_window)[0]
            if to_drop.size:
                start += to_drop[-1] + 1
                continue

        out_X.append(X[start:end])
        out_y.append(target[end:end + target_size])
        X_index.append(index[start])
        y_index.append(index[end])
        start = start + step_size

    return np.asarray(out_X), np.asarray(out_y), np.asarray(X_index), np.asarray(y_index)

X, y, X_index, y_index = rolling_window_sequences(X, index, 
                                                  window_size=100, 
                                                  target_size=1, 
                                                  step_size=1,
                                                  target_column=0)


Вывод размерности данных и самих окон для наглядности

In [ ]:
print("Training data input shape: {}".format(X.shape))
print("Training data index shape: {}".format(X_index.shape))
print("Training y shape: {}".format(y.shape))
print("Training y index shape: {}".format(y_index.shape))
plot_rws(X)


Обучение модели TadGAN и установка гипепарамеров(https://arxiv.org/abs/2009.07769)

In [ ]:
hyperparameters["epochs"] = 3
hyperparameters["input_shape"] = (100, 1) # based on the window size
hyperparameters["optimizer"] = "tensorflow.keras.optimizers.Adam"
hyperparameters["learning_rate"] = 0.0005
hyperparameters["latent_dim"] = 20
hyperparameters["batch_size"] = 64

tgan = TadGAN(**hyperparameters)
tgan.fit(X)

Предсказывание данных и просмотр получившихся окон

In [ ]:
X_hat, critic = tgan.predict(X)

# visualize X_hat
plot_rws(X_hat)


Вывод оригиналиного и сгенерированного графиков

In [ ]:
y_hat = unroll_ts(X_hat)

# plot the time series
plot_ts([y, y_hat], labels=['original', 'reconstructed'])


Вычисление ошибки генерации

In [ ]:
error = np.zeros(shape=y.shape)
length = y.shape[0]
for i in range(length):
    error[i] = abs(y_hat[i] - y[i])

# visualize the error curve
fig = plt.figure(figsize=(30, 3))
plt.plot(error)
plt.show()


Построение графиков: показания с датчика, предсказанные данные датчика поверх снятых с датчика данных, графики вероятности аномалий(на втором графике хорошо видно совокупности аномалий, на третьем графике хорошо видно точечные аномалии)

In [ ]:

error_aggregate, true_index, true, pred = score_anomalies(X, X_hat, critic, X_index, rec_error_type="dtw", comb="mult")
error_pinpoints, _, _, _ = score_anomalies(X, X_hat, critic, X_index, rec_error_type="dtw", comb="mult", smooth=False)
pred = np.array(pred).mean(axis=2)


Отображение графиков

In [ ]:
plot_error([[true, pred], error_pinpoints])
plot_error([[true, pred], error_aggregate])